# Cycle Manager Helper
Functions for creating and managing cycles.
Run this with: `%run /home/jovyan/work/system/helpers/02_CycleManager.ipynb`

In [ ]:
# Load dependencies
if 'execute_query' not in globals():
    %run /workspace/system/helpers/01_Database.ipynb

import shutil
from datetime import datetime

In [ ]:
def get_existing_cycle_names() -> List[str]:
    """Get all existing cycle names from database and filesystem."""
    names = set()
    
    # Get from database
    df = execute_query("SELECT cycle_name FROM irp_cycle")

    # Check if dataframe is empty
    if df.empty:
        return []

    names.update(df['cycle_name'].tolist())
    
    # Get from filesystem
    for path in WORKFLOWS_PATH.iterdir():
        if path.is_dir():
            if path.name.startswith('Active_'):
                names.add(path.name[7:])  # Remove 'Active_' prefix
            elif path != ARCHIVE_PATH and not path.name.startswith('_'):
                names.add(path.name)
    
    # Check archive
    if ARCHIVE_PATH.exists():
        for path in ARCHIVE_PATH.iterdir():
            if path.is_dir():
                names.add(path.name)
    
    return sorted(list(names))

def validate_cycle_name(name: str) -> Tuple[bool, str]:
    """Validate a cycle name."""
    if not name:
        return False, "Cycle name cannot be empty"
    
    if len(name) > 100:
        return False, "Cycle name must be 100 characters or less"
    
    # Check for invalid characters
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|']
    for char in invalid_chars:
        if char in name:
            return False, f"Cycle name cannot contain '{char}'"
    
    # Check if name already exists
    existing = get_existing_cycle_names()
    if name in existing:
        return False, f"Cycle name '{name}' already exists"
    
    return True, "Valid"

def archive_active_cycle() -> Optional[str]:
    """Archive the current active cycle."""
    # Find active cycle directory
    active_dir = None
    for path in WORKFLOWS_PATH.iterdir():
        if path.is_dir() and path.name.startswith('Active_'):
            active_dir = path
            break
    
    if not active_dir:
        return None
    
    # Extract cycle name
    cycle_name = active_dir.name[7:]  # Remove 'Active_' prefix
    
    # Create archive directory if needed
    ARCHIVE_PATH.mkdir(exist_ok=True)
    
    # Move to archive
    archive_dest = ARCHIVE_PATH / cycle_name
    shutil.move(str(active_dir), str(archive_dest))
    
    # Update database
    query = """
    UPDATE irp_cycle 
    SET status = 'archived', archived_ts = NOW() 
    WHERE cycle_name = %s AND status = 'active'
    """
    execute_command(query, (cycle_name,))
    
    # Clear system lock
    execute_command("UPDATE irp_system_lock SET active_cycle_id = NULL, locked_by = NULL, locked_at = NULL WHERE id = 1")
    
    print(f"📦 Archived cycle: {cycle_name}")
    return cycle_name

def create_new_cycle(cycle_name: str) -> bool:
    """Create a new cycle with the given name."""
    # Validate name
    valid, message = validate_cycle_name(cycle_name)
    if not valid:
        print(f"❌ {message}")
        return False
    
    # Archive existing active cycle
    archived = archive_active_cycle()
    if archived:
        print(f"✅ Previous cycle '{archived}' archived")
    
    # Create new cycle in database
    query = "INSERT INTO irp_cycle (cycle_name, status, created_by) VALUES (%s, 'active', %s)"
    execute_command(query, (cycle_name, SYSTEM_USER))
    
    # Get the new cycle ID
    cycle_id = execute_scalar("SELECT id FROM irp_cycle WHERE cycle_name = %s", (cycle_name,))
    
    # Set as active cycle
    execute_command(
        "UPDATE irp_system_lock SET active_cycle_id = %s, locked_by = %s, locked_at = NOW() WHERE id = 1",
        (cycle_id, SYSTEM_USER)
    )
    
    # Create directory structure
    active_dir = WORKFLOWS_PATH / f"Active_{cycle_name}"
    
    # Copy from template
    if TEMPLATE_PATH.exists():
        shutil.copytree(TEMPLATE_PATH, active_dir)
        print(f"📁 Created directory: {active_dir}")
        
        # Register stages and steps from template
        register_stages_and_steps(cycle_id, active_dir)
    else:
        # Create basic structure
        active_dir.mkdir(exist_ok=True)
        (active_dir / 'notebooks').mkdir(exist_ok=True)
        (active_dir / 'files').mkdir(exist_ok=True)
        (active_dir / 'logs').mkdir(exist_ok=True)
        print(f"📁 Created basic directory structure: {active_dir}")
    
    print(f"✅ Cycle '{cycle_name}' created successfully")
    return True

def register_stages_and_steps(cycle_id: int, cycle_dir: Path):
    """Register stages and steps from directory structure."""
    notebooks_dir = cycle_dir / 'notebooks'
    if not notebooks_dir.exists():
        return
    
    # Find all stage directories
    stages = sorted([d for d in notebooks_dir.iterdir() if d.is_dir() and d.name.startswith('Stage_')])
    
    for stage_dir in stages:
        # Parse stage number and name
        parts = stage_dir.name.split('_', 2)
        if len(parts) >= 2:
            stage_num = int(parts[1]) if parts[1].isdigit() else 0
            stage_name = parts[2] if len(parts) > 2 else stage_dir.name
            
            # Insert stage
            query = "INSERT INTO irp_stage (cycle_id, stage_num, stage_name) VALUES (%s, %s, %s)"
            execute_command(query, (cycle_id, stage_num, stage_name))
            stage_id = execute_scalar("SELECT id FROM irp_stage WHERE cycle_id = %s AND stage_num = %s", 
                                     (cycle_id, stage_num))
            
            # Find all step notebooks
            steps = sorted([f for f in stage_dir.glob('Step_*.ipynb')])
            
            for step_file in steps:
                # Parse step number and name
                step_parts = step_file.stem.split('_', 2)
                if len(step_parts) >= 2:
                    step_num = int(step_parts[1]) if step_parts[1].isdigit() else 0
                    step_name = step_parts[2] if len(step_parts) > 2 else step_file.stem
                    
                    # Insert step
                    query = """INSERT INTO irp_step 
                              (stage_id, step_num, step_name, notebook_path) 
                              VALUES (%s, %s, %s, %s)"""
                    notebook_path = str(step_file.relative_to(cycle_dir))
                    execute_command(query, (stage_id, step_num, step_name, notebook_path))
    
    print(f"✅ Registered {len(stages)} stages with steps")

def get_cycle_status() -> pd.DataFrame:
    """Get status of all cycles."""
    query = """
    SELECT 
        c.cycle_name,
        c.status,
        c.created_ts,
        c.archived_ts,
        (SELECT COUNT(*) FROM irp_stage WHERE cycle_id = c.id) as stages,
        (SELECT COUNT(*) FROM irp_step st 
         INNER JOIN irp_stage sg ON st.stage_id = sg.id 
         WHERE sg.cycle_id = c.id) as steps
    FROM irp_cycle c
    ORDER BY c.created_ts DESC
    """
    return execute_query(query)

print("✅ Cycle Manager loaded")